In [1]:
!wget http://www.aueb.gr/users/ion/data/lingspam_public.tar.gz

--2021-09-04 02:45:32--  http://www.aueb.gr/users/ion/data/lingspam_public.tar.gz
Resolving www.aueb.gr (www.aueb.gr)... 195.251.255.156
Connecting to www.aueb.gr (www.aueb.gr)|195.251.255.156|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www2.aueb.gr/users/ion/data/lingspam_public.tar.gz [following]
--2021-09-04 02:45:32--  http://www2.aueb.gr/users/ion/data/lingspam_public.tar.gz
Resolving www2.aueb.gr (www2.aueb.gr)... 195.251.255.138
Connecting to www2.aueb.gr (www2.aueb.gr)|195.251.255.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11564714 (11M) [application/x-gzip]
Saving to: ‘lingspam_public.tar.gz’

lingspam_public.tar 100%[===================>]  11.03M   743KB/s    in 11s     

2021-09-04 02:45:44 (1005 KB/s) - ‘lingspam_public.tar.gz’ saved [11564714/11564714]



In [2]:
!tar -zxf ./lingspam_public.tar.gz

In [3]:
import os
import glob
import pandas as pd

path = "./lingspam_public/bare/"

text = []
label = []

# part?ディレクトリ配下にあるメールデータを読み込み、迷惑メールとそうでないものを分別してラベルを追加
for part in range(1,10):
    folder = os.path.join(path, 'part'+str(part))
    for filePath in glob.glob(os.path.join(folder, '*.txt')):
        if ('spmsga' in filePath):
            with open(filePath) as f:
                text.append(f.read())
                label.append(1)
        else:
            with open(filePath) as f:
                text.append(f.read())
                label.append(0)

In [4]:
data = pd.DataFrame()
data['Text'] = text
data['label'] = label

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizerを初期化する。stop_wordsにenglishを指定し、一般的な単語を除外する
tfidf = TfidfVectorizer(stop_words="english")

X = tfidf.fit_transform(data['Text'])
column_names = tfidf.get_feature_names()

# Xにベクトル化した値を整形して代入
X = pd.DataFrame(X.toarray())
X = X.astype('float')
# カラム名を設定
X.columns = column_names
y = data['label']

In [6]:
X

,00,000,0000,00001,00003000140,00003003958,0001,00010,00014,0003,00036,000bp,000s,000yen,001,0010,0010010034,0011,00133,0014,00170,0019,00198,002,002656,0027,003,0030,0031,00333,0037,0039,003n7,004,0041,0049,005,0057,006,0067,...,zuordnung,zur,zurab,zurich,zurif,zuruckzusenden,zurueck,zusaetzlich,zusagen,zusammenarbeit,zusammengest,zusammenhang,zusammenstellung,zuzanna,zvegintseva,zvjezdana,zwaan,zwanenburg,zwanzig,zwart,zwarts,zweck,zwecke,zwei,zweig,zweigenbaum,zweisprachigen,zweiter,zweitsprache,zwicky,zwischen,zwitserlood,zxgah7qabjh,zybatov,zybatow,zygmunt,zyokyoozyu,zytkow,zz214,zzlsa
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2598,0.160656,0.186365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2599,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2600,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
!pip install optuna

     |████████████████████████████████| 302 kB 2.7 MB/s 
     |████████████████████████████████| 80 kB 7.7 MB/s 
     |████████████████████████████████| 208 kB 32.4 MB/s 
     |████████████████████████████████| 75 kB 3.3 MB/s 
     |████████████████████████████████| 111 kB 57.2 MB/s 
     |████████████████████████████████| 141 kB 59.1 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=e2ea5321f274b4c3a6ce4ac1a22f459f769b55b69012b2cab4de49c12844c473
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [8]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as olgb
import optuna

# データセットを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=101)

# LightGBM用のデータセットに変換
train = olgb.Dataset(X_train, y_train)

# パラメータの設定
params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

# 交差検証を使用したハイパーパラメータの探索
tuner = olgb.LightGBMTunerCV(params, train, num_boost_round=100)

# ハイパーパラメータ探索の実行
tuner.run()

[I 2021-09-04 02:46:05,261] A new study created in memory with name: no-name-f3370eba-0ea9-46ef-901d-dafa497fa204
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	cv_agg's binary_logloss: 0.19862 + 0.00359126
[2]	cv_agg's binary_logloss: 0.185386 + 0.00389778
[3]	cv_agg's binary_logloss: 0.17502 + 0.00253609
[4]	cv_agg's binary_logloss: 0.168119 + 0.00296702
[5]	cv_agg's binary_logloss: 0.160594 + 0.00332557
[6]	cv_agg's binary_logloss: 0.155014 + 0.0034501
[7]	cv_agg's binary_logloss: 0.151124 + 0.00310321
[8]	cv_agg's binary_logloss: 0.146432 + 0.00249513
[9]	cv_agg's binary_logloss: 0.143352 + 0.00252627
[10]	cv_agg's binary_logloss: 0.140082 + 0.00314259
[11]	cv_agg's binary_logloss: 0.137587 + 0.00357983
[12]	cv_agg's binary_logloss: 0.135211 + 0.00386379
[13]	cv_agg's binary_logloss: 0.133094 + 0.00369616
[14]	cv_agg's binary_logloss: 0.13021 + 0.00428139
[15]	cv_agg's binary_logloss: 0.128622 + 0.00540759
[16]	cv_agg's binary_logloss: 0.127512 + 0.00589657
[17]	cv_agg's binary_logloss: 0.126169 + 0.00604162
[18]	cv_agg's binary_logloss: 0.125014 + 0.00593181
[19]	cv_agg's binary_logloss: 0.123528 + 0.0064411
[20]	cv_agg's binary_loglo

feature_fraction, val_score: 0.161792:  14%|#4        | 1/7 [00:22<02:13, 22.18s/it]

[100]	cv_agg's binary_logloss: 0.161792 + 0.0132758
[1]	cv_agg's binary_logloss: 0.20034 + 0.00454703
[2]	cv_agg's binary_logloss: 0.185328 + 0.00449707
[3]	cv_agg's binary_logloss: 0.17625 + 0.0050094
[4]	cv_agg's binary_logloss: 0.168638 + 0.00685772
[5]	cv_agg's binary_logloss: 0.161974 + 0.00709315
[6]	cv_agg's binary_logloss: 0.157467 + 0.00738225
[7]	cv_agg's binary_logloss: 0.152454 + 0.00726126
[8]	cv_agg's binary_logloss: 0.148931 + 0.00803833
[9]	cv_agg's binary_logloss: 0.14606 + 0.00861167
[10]	cv_agg's binary_logloss: 0.143069 + 0.00893744
[11]	cv_agg's binary_logloss: 0.140713 + 0.00943345
[12]	cv_agg's binary_logloss: 0.138444 + 0.00977096
[13]	cv_agg's binary_logloss: 0.13571 + 0.00982357
[14]	cv_agg's binary_logloss: 0.13467 + 0.0103296
[15]	cv_agg's binary_logloss: 0.133473 + 0.0111036
[16]	cv_agg's binary_logloss: 0.131479 + 0.0111776
[17]	cv_agg's binary_logloss: 0.129984 + 0.0120702
[18]	cv_agg's binary_logloss: 0.129257 + 0.012519
[19]	cv_agg's binary_logloss: 0.1

feature_fraction, val_score: 0.161792:  29%|##8       | 2/7 [00:46<01:58, 23.64s/it]

...

[1]	cv_agg's binary_logloss: 0.208014 + 0.00416568
[2]	cv_agg's binary_logloss: 0.199072 + 0.00450656
[3]	cv_agg's binary_logloss: 0.191381 + 0.0054987
[4]	cv_agg's binary_logloss: 0.182709 + 0.00463726
[5]	cv_agg's binary_logloss: 0.176651 + 0.00514968
[6]	cv_agg's binary_logloss: 0.170571 + 0.00402081
[7]	cv_agg's binary_logloss: 0.166893 + 0.00494146
[8]	cv_agg's binary_logloss: 0.162827 + 0.00498671
[9]	cv_agg's binary_logloss: 0.158585 + 0.00456322
[10]	cv_agg's binary_logloss: 0.155281 + 0.00367659
[11]	cv_agg's binary_logloss: 0.151993 + 0.00310774
[12]	cv_agg's binary_logloss: 0.14952 + 0.00280558
[13]	cv_agg's binary_logloss: 0.147405 + 0.0023548
[14]	cv_agg's binary_logloss: 0.145085 + 0.00194703
[15]	cv_agg's binary_logloss: 0.143357 + 0.00154736
[16]	cv_agg's binary_logloss: 0.140984 + 0.00186524
[17]	cv_agg's binary_logloss: 0.139922 + 0.00189268
[18]	cv_agg's binary_logloss: 0.138671 + 0.00184566
[19]	cv_agg's binary_logloss: 0.137266 + 0.0016182
[20]	cv_agg's binary_logl

min_data_in_leaf, val_score: 0.113321:  80%|########  | 4/5 [00:41<00:09,  9.62s/it]

[1]	cv_agg's binary_logloss: 0.207671 + 0.00352323
[2]	cv_agg's binary_logloss: 0.197988 + 0.0037172
[3]	cv_agg's binary_logloss: 0.188933 + 0.00380375
[4]	cv_agg's binary_logloss: 0.180954 + 0.00391493
[5]	cv_agg's binary_logloss: 0.176094 + 0.00482106
[6]	cv_agg's binary_logloss: 0.171508 + 0.00470522
[7]	cv_agg's binary_logloss: 0.165607 + 0.00458019
[8]	cv_agg's binary_logloss: 0.161625 + 0.00497825
[9]	cv_agg's binary_logloss: 0.15667 + 0.0047023
[10]	cv_agg's binary_logloss: 0.153066 + 0.00491583
[11]	cv_agg's binary_logloss: 0.149871 + 0.00442856
[12]	cv_agg's binary_logloss: 0.147017 + 0.00435266
[13]	cv_agg's binary_logloss: 0.144814 + 0.00468135
[14]	cv_agg's binary_logloss: 0.142315 + 0.0046972
[15]	cv_agg's binary_logloss: 0.139796 + 0.00433135
[16]	cv_agg's binary_logloss: 0.138077 + 0.00385349
[17]	cv_agg's binary_logloss: 0.136406 + 0.00395715
[18]	cv_agg's binary_logloss: 0.135267 + 0.00413744
[19]	cv_agg's binary_logloss: 0.134216 + 0.00449893
[20]	cv_agg's binary_logl

min_data_in_leaf, val_score: 0.113321: 100%|##########| 5/5 [00:49<00:00,  9.86s/it]

[99]	cv_agg's binary_logloss: 0.116664 + 0.00917598
[100]	cv_agg's binary_logloss: 0.116509 + 0.0093748


In [9]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

# 訓練データとテストデータを設定
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# ハイパーパラメータ探索で特定した値を設定(このパラメータは参考値)
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'lambda_l1': tuner.best_params['lambda_l1'],
    'lambda_l2': tuner.best_params['lambda_l2'],
    'num_leaves': tuner.best_params['num_leaves'],
    'feature_fraction': tuner.best_params['feature_fraction'],
    'bagging_fraction': tuner.best_params['bagging_fraction'],
    'bagging_freq': tuner.best_params['bagging_freq'],
    'min_child_samples': tuner.best_params['min_child_samples']
}

# 訓練の実施
gbm = lgb.train(
    params,
    train_data,
    num_boost_round=100,
    verbose_eval=0,
)

# テスト用データを使って予測する
preds = gbm.predict(X_test)
# 返り値は確率になっているので四捨五入する
pred_labels = np.rint(preds)
# 予測精度と混同行列の出力
print("Accurary: {:.5f} %".format(100 * accuracy_score(y_test, pred_labels)))
print(confusion_matrix(y_test, pred_labels))

Accurary: 93.66603 %
[[468   9]
 [ 24  20]]
